In [1]:
!pip install pdfminer.six pillow matplotlib python-dotenv anthropic tiktoken
from google.colab import files
from pdfminer.high_level import extract_text
from PIL import Image
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 16.3 MB/s eta 0:00:00


In [9]:
uploaded = files.upload()

pdf_path = list(uploaded.keys())[0]
# pdf_path

brand_text = extract_text(pdf_path)

Saving Brand Kit Template.pdf to Brand Kit Template (2).pdf


In [10]:
brand_text

'1. Brand Overview \nBrand Name: KiboSF \nBrand Essence: Modern, trustworthy, and innovative. \nPurpose: Deliver clean, consistent visuals aligned with a unified creative identity. \nTone: Minimal, confident, and human‑centered. \n\n2. Logo Usage Guidelines \nPrimary Logo \nUse the primary logo on light or white backgrounds. \n\nClear Space \nMaintain at least 1× the logo height of clear padding on all sides. \n\nMinimum Size \nDigital: 24px height \n\nPrint: 0.5 inches \n\nAvoid \nStretching or skewing \n\nChanging colors outside the approved palette \n\nAdding shadows, bevels, or glows \n\n#1A73E8 \n#4A5568 \n\n3. Color Palette \nPrimary Colors \nColor Name  Hex  RGB \nBrand Blue \nSlate Gray \nWhite  #FFFFFF \nSecondary Colors \nColor Name  Hex  RGB \nSoft Sky \nMist Gray \nOcean Ink \n4. Typography Guidelines \nPrimary Typeface \nInter \n\n#A7C5F9 \n#CBD5E0 \n#0F3D91 \n\n26, 115, 232 \n74, 85, 104 \n\n255, 255, 255 \n\n167, 197, 249 \n203, 213, 224 \n15, 61, 145 \n\nWeights: Regula

In [4]:
def extract_colors(img_path, num_colors=5):
    img = Image.open(img_path).convert("RGB")
    img_arr = np.array(img).reshape((-1,3))
    counter = Counter(map(tuple, img_arr))
    most_common = counter.most_common(num_colors)
    hex_colors = ['#%02x%02x%02x' % c[0] for c in most_common]
    return hex_colors

# Uncomment and upload an image if available
# uploaded_img = files.upload()
# img_path = list(uploaded_img.keys())[0]
# extract_colors(img_path)

In [5]:
FIBO_BRAND_SCHEMA = {
    "color_palette": [],
    "lighting_style": "",
    "mood": "",
    "camera": {
        "angle": "",
        "lens": "",
        "fov": ""
    },
    "composition": "",
    "texture_style": "",
    "brand_keywords": [],
}

In [ ]:
import os 
import dotenv
dotenv.load_dotenv()

In [ ]:
from anthropic import Anthropic
client = Anthropic(api_key="CLAUDE_API_KEY")
import json

prompt = f"""
You are an assistant that converts brand guidelines into structured JSON parameters
for Bria FIBO image generation.

Here is the brand guideline text:

{brand_text}

Extract ONLY:
- brand color palette (hex or descriptive)
- lighting style
- mood/visual tone
- camera preferences (angles, lenses, FOV if implied)
- composition rules
- textures/material preferences
- 5–10 core brand keywords

Return JSON that matches this schema exactly:

{json.dumps(FIBO_BRAND_SCHEMA, indent=2)}
"""

response = client.messages.create(
    model="claude-haiku-4-5-20251001",
    max_tokens=1000,
    temperature=0,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

raw_json = response.content[0].text
print(raw_json)

```json
{
  "color_palette": [
    "#1A73E8",
    "#4A5568",
    "#FFFFFF",
    "#A7C5F9",
    "#CBD5E0",
    "#0F3D91"
  ],
  "lighting_style": "Bright, minimal with strong natural light and soft shadows",
  "mood": "Modern, trustworthy, innovative, minimal, confident, human-centered, professional, warm but authoritative",
  "camera": {
    "angle": "Clean, straightforward compositions",
    "lens": "Not specified",
    "fov": "Not specified"
  },
  "composition": "Clean compositions with realism and authenticity. Avoid overly stylized filters, heavy saturation, and busy backgrounds. Prioritize clarity and direct visual communication.",
  "texture_style": "Minimal, clean surfaces. Geometric and line-based aesthetic with rounded edges. Avoid heavy textures or ornamental details.",
  "brand_keywords": [
    "modern",
    "trustworthy",
    "innovative",
    "minimal",
    "confident",
    "human-centered",
    "clean",
    "authentic",
    "geometric",
    "professional"
  ]
}
```


In [15]:
import json, re

def extract_and_fix_json(raw_text: str):
    """
    Cleans messy LLM output and extracts a valid JSON object.
    Handles:
      - Markdown code fences
      - Comments
      - Trailing commas
      - Natural language before/after JSON
      - Extra whitespace
    """

    # 1. Remove code fences
    cleaned = raw_text.replace("```json", "").replace("```", "").strip()

    # 2. Remove comments like // something
    cleaned = re.sub(r"//.*", "", cleaned)

    # 3. Try to detect the JSON object inside the text
    #    Finds the first {...} block
    match = re.search(r"\{[\s\S]*\}", cleaned)
    if match:
        cleaned = match.group(0)

    # 4. Remove trailing commas before } or ]
    cleaned = re.sub(r",\s*([}\]])", r"\1", cleaned)

    # 5. Fix single quotes → double quotes (safe case)
    if "'" in cleaned and '"' not in cleaned:
        cleaned = cleaned.replace("'", '"')

    # 6. Final load
    try:
        return json.loads(cleaned)
    except Exception as e:
        print("❌ Still invalid JSON. Debug output below:")
        print(cleaned)
        raise e

In [16]:
try:
    brand_params = extract_and_fix_json(raw_json)
    print("JSON loaded successfully!")
    print(json.dumps(brand_params, indent=2))
except:
    print("JSON parse failed — manual fix needed.")

JSON loaded successfully!
{
  "color_palette": [
    "#1A73E8",
    "#4A5568",
    "#FFFFFF",
    "#A7C5F9",
    "#CBD5E0",
    "#0F3D91"
  ],
  "lighting_style": "Bright, minimal with strong natural light and soft shadows",
  "mood": "Modern, trustworthy, innovative, minimal, confident, human-centered, professional, warm but authoritative",
  "camera": {
    "angle": "Clean, straightforward compositions",
    "lens": "Not specified",
    "fov": "Not specified"
  },
  "composition": "Clean compositions with realism and authenticity. Avoid overly stylized filters, heavy saturation, and busy backgrounds. Prioritize clarity and direct visual communication.",
  "texture_style": "Minimal, clean surfaces. Geometric and line-based aesthetic with rounded edges. Avoid heavy textures or ornamental details.",
  "brand_keywords": [
    "modern",
    "trustworthy",
    "innovative",
    "minimal",
    "confident",
    "human-centered",
    "clean",
    "authentic",
    "geometric",
    "professiona

In [17]:
with open("brand_lockfile.json", "w") as f:
    json.dump(brand_params, f, indent=2)

files.download("brand_lockfile.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>